In [1]:
import torch
import numpy as np
import time
import DRLPDE_nn


In [7]:
class IncompressibleNN(torch.nn.Module):
    
    ### Incompressible neural network
    ### curl operation built in
    
    def __init__(self, depth, width, x_dim, is_unsteady, num_param, **nn_param):
        super(IncompressibleNN, self).__init__()
        
        self.x_dim = x_dim
        self.input_dim = self.x_dim + is_unsteady + num_param
        
        self.dim_out = 1
        
        modules = []
        modules.append(torch.nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(torch.nn.Linear(depth, depth))
            modules.append(torch.nn.Tanh())
        modules.append(torch.nn.Linear(depth, self.dim_out))
                       
        self.sequential_model = torch.nn.Sequential(*modules)
    
    def curl(self, a, x):

        dadx = torch.autograd.grad(a, x, grad_outputs = torch.ones_like(a), 
                                    create_graph = True, retain_graph = True, allow_unused=False)[0]

        u = torch.stack([dadx[:,1], -dadx[:,0]] , dim=1)
                   
        return u
    
    def forward(self, x):
        a = self.sequential_model(x)
        u = self.curl(a, x)
            
        return u

In [8]:
model = IncompressibleNN(4, 20, 2, False, 0)

X = torch.randn((10,2), requires_grad=True)
U = model(X)


In [10]:
e = torch.eye(1)

Ux = torch.autograd.grad(U, X, grad_outputs=e[0,:].repeat(U.size(0), 1), create_graph=True, retain_graph=True)[0]

In [16]:
Ux.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
loss = torch.norm(Target, dim=1)

### Rejection sampling ###
# generate uniform( max_loss )
# Compare sample < uniform
# Then resample

check_sample = torch.rand(X.size(0), device=X.device)
resample_index = loss < check_sample

loss = torch.mean( loss )

In [6]:
loss.is_leaf

False

In [10]:
class Data(torch.utils.data.Dataset):
    
    def __init__(self, num):
        
        X = torch.randn(num,2)
        #region = close_to_region(Xold)
        
        self.location = X
        self.num_pts = num
        #self.region = region
        
    def __len__(self):
        ### How many data points are there?
        return self.num_pts
    
    def __getitem__(self, index):
        ### Gets one sample of data
        ### 
        return self.location[index,:], index


In [56]:
X = Data(10)

Batch = torch.utils.data.DataLoader(X, batch_size=2, shuffle=True)

In [61]:
for Xbatch, index in Batch:
    print(index.size())
    print(X.location[0])

torch.Size([2])
tensor([ 1.0808, -0.9571])
torch.Size([2])
tensor([ 1.0808, -0.9571])
torch.Size([2])
tensor([ 1.0808, -0.9571])
torch.Size([2])
tensor([ 1.0808, -0.9571])
torch.Size([2])
tensor([ 1.0808, -0.9571])


In [52]:
index[ torch.randn(2) < 0 ]

tensor([0])

In [39]:
torch.cat(( index, torch.tensor( [1024] , dtype=torch.int8) ))

tensor([0, 8, 0])

In [5]:
B = torch.tensor( [1024], dtype=torch.int64)

In [6]:
B = B.to('cpu')